# Scrape models info from Hugging face

In [19]:
import requests
from bs4 import BeautifulSoup
import json

Output:
- List of categories
- List of models inside each category
- Model:
    + Name
    + ...

In [20]:
#define list of tasks
categories = [] #'Multimodal', 'Computer Vision', 'Natural Language Processing', 'Audio', 'Tabular', 'Reinforcement Learning'
cat_map = {}    #key: category, value: list of tasks


In [21]:
def custom_query(get_url):
    #print(get_url)
    try:
        r = requests.get(get_url)
        return r.json()
    except Exception as e:
       print(e)
       return {'error': e}

In [22]:
#find task statistics
def find_task_details(tag):
    simple_details = {}
    details = custom_query('https://huggingface.co/models-json?sort=downloads&withCount=true&pipeline_tag=' + tag)
    if 'error' in details:
        return {}   #error
    #extract basic info
    simple_details['numTotalItems'] = details['numTotalItems']
    if 'models' in details:
        model_details = []
        for model in details['models']:
            model_details.append({
                'id': model['id'],
                'downloads': model['downloads']
            })
        simple_details['models'] = model_details
    return simple_details

In [28]:
list_url = 'https://huggingface.co/models?sort=trending'
response = requests.get(list_url)

soup = BeautifulSoup(response.content, 'html.parser')
items = soup.find_all('div', attrs={'class': 'mb-3'})
index = 0
for item in items:
    a_tags = item.find_all('a', attrs={'class': 'mb-1 mr-1 md:mb-1.5 md:mr-1.5 rounded-lg'})
    if a_tags != None and len(a_tags) > 0:
        cat_tag = item.find('div', attrs={'class': 'mb-3 text-sm font-medium text-gray-500'})
        cat_name = cat_tag.text.strip().replace('\n', "").replace('\t', "")
        categories.append(cat_name)
        task_list = []
        for a_tag in a_tags:
            # if index > 0:
            #     break
            tag_name = a_tag['href'].replace('/models?pipeline_tag=', '')
            task_details = find_task_details(tag_name)
            task_list.append({
                'tag': tag_name,
                'name': a_tag.find('span').text,
                'task_details': task_details
            })
            index += 1
        cat_map[cat_name+''] = task_list
        # if index > 0:
        #     break

#done (1m 40s)
#print(cat_map)
with open('huggingface_modellist.json', 'w') as f:
    f.write(str(cat_map).replace("'", '"'))


In [ ]:
#find details of each tasks